### KoGPT2 Chatbot

- 언어 모델 (Language Model)이란 문장 혹은 단어에 확률을 할당하여 컴퓨터가 처리할 수 있도록 하는 모델
- 최근 머신러닝 모델들은 백지부터 시작하는 경우가 거의 없음
- 전이 학습 (Transfer Learning)은 자연어 처리 분야에 있어서 경이로운 발전

#### BERT (Bidirectional Encoder Representations from Transformers)

- 2019년 10월 25일 구글 리서치 팀에 의해 공개된 자연어처리 사전 훈련 모델
- BERT는 인코더만 존재
- BERT 모델은 100여개가 넘는 언어 학습을 지원. BERT-Base, BERT-Large, BERT-Base, Multilingual, 그리고 BERT-Base, Chinese 모델
    - 모델 뒤에 Cased와 Uncased가 붙혀져 있는데, Uncased의 경우 대소문자 구분을 하지 않는 모델
- 세부적인 과제를 수행하도록 파인튜닝(fine-tuning) 작업이 필요

#### GPT

- 일론 머스크와 샘 알트만이 설립한 openAI에서 개발한 자연어 처리 모델
- 2019년 GPT-2의 공개 후 엄청난 성능으로 많은 사람들을 놀라게 하였고, 2020년 6월에는 GPT-3로 인공지능계에 돌풍
- GPT-2는 주어진 텍스트의 다음 단어를 잘 예측할 수 있도록 학습된 언어모델이며 문장 생성에 최적화

##### KoGPT2
- KoGPT2는 부족한 한국어 성능을 극복하기 위해 40GB 이상의 텍스트로 학습된 한국어 디코더(decoder) 언어모델
- 한국어 위키 백과 이외, 뉴스, 모두의 말뭉치 v1.0, 청와대 국민청원 등의 다양한 데이터를 학습시켜 만든 언어모델
- 논문 Attention Is All You Need에서 제시한 인코더+디코더 구조에서 인코더 블록을 제거하고 디코더 블록만 사용한 모델

#### 한국어 챗봇 Dataset

- https://github.com/songys/Chatbot_data 데이터 사용
    - 데이터의 Q 필드를 발화, A 필드를 발화 그리고 감정 레이블을 사용
    - 감정 레이블은 label에 정의된 일상다반사 0, 이별(부정) 1, 사랑(긍정) 2)를 그대로 적용

In [3]:
!pip install transformers

     ---------------------------------------- 0.0/44.4 kB ? eta -:--:--
     --------- ------------------------------ 10.2/44.4 kB ? eta -:--:--
     -------------------------------------- 44.4/44.4 kB 437.8 kB/s eta 0:00:00
  Using cached huggingface_hub-0.27.1-py3-none-any.whl.metadata (13 kB)
  Using cached regex-2024.11.6-cp311-cp311-win_amd64.whl.metadata (41 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached tokenizers-0.21.0-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
  Using cached safetensors-0.5.2-cp38-abi3-win_amd64.whl.metadata (3.9 kB)
  Using cached charset_normalizer-3.4.1-cp311-cp311-win_amd64.whl.metadata (36 kB)
  Using cached urllib3-2.3.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2024.12.14-py3-none-any.whl.metadata (2.3 kB)
   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   - -------------------------------------- 0.3/9.7 MB 9.6 MB/s eta 0:00:01
   --------- ------------------------------ 2.2/9.7 


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import numpy as np
import pandas as pd
import torch
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.core import LightningModule
from torch.utils.data import DataLoader, Dataset
from transformers.optimization import AdamW, get_cosine_schedule_with_warmup
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel
import re

d:\01_Programming\100_HugoBank\Mine\study-pytorch\torch-cpu-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


- 챗봇 데이터를 다운로드

In [8]:
import urllib.request

urllib.request.urlretrieve(
    "https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv",
    filename="ChatBotData.csv",
)
Chatbot_Data = pd.read_csv("ChatBotData.csv")
# Test 용으로 300개 데이터만 처리한다.
Chatbot_Data = Chatbot_Data[:300]
Chatbot_Data.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


- Tokenizer는 모델에 어떠한 입력을 넣어주기 위해서 전처리를 담당
    - 허깅페이스의 PreTrainedTokenizer 인 GPT2Tokenizer 를 사용
- Tokenizer의 기능
    1. Tokenizing : 입력 문자열을 token id로 변환(encoding), token id를 다시 문자열로 변환(decoding)의 기능
    2. 기존의 구조(BPE, Sentencepiece 등)에 독립적으로 추가적인 token들을 추가하는 기능
    3. Special token들을 (mask, BOS, EOS 등) 관리하는 기능

In [6]:
Q_TKN = "<usr>"
A_TKN = "<sys>"
BOS = '<s>'
EOS = '</s>'
MASK = '<unused0>'
SENT = '<unused1>'
PAD = '<pad>'

In [7]:
koGPT2_TOKENIZER = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
            bos_token=BOS, eos_token=EOS, unk_token='<unk>',
            pad_token=PAD, mask_token=MASK) 
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
d:\01_Programming\100_HugoBank\Mine\study-pytorch\torch-cpu-env\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\perso\.cache\huggingface\hub\models--skt--kogpt2-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode 

- 파라미터 의미
    - bos_token : 문장의 시작을 나타내는 token
    - eos_token : 문장의 끝을 나타내는 token
    - unk_token : 모르는 단어를 나타내는 token
    - pad_token : 동일한 batch 내에서 입력의 크기를 동일하게 하기 위해서 사용해는 token

    

- PreTrainedTokenizer 에서 제공되는 함수
    - tokenize() : tokenizer를 이용해서 string을 token id의 리스트로 변환
    - get_added_vocab() : token to index에 해당하는 dict를 리턴
    - batch_decode() : token id로 구성된 입력을 하나의 연결된 string으로 출력
    - convert_ids_to_tokens() : token id 의 리스트를 token으로 변환. skip_special_tokens=True로 하면 decoding할 때 special token들을 제거
    - convert_tokens_to_ids() : token string의 리스트를 token id 또는 Token id의 리스트로 변환
    - decode() : tokenizer 와 vocabulary를 이용해서 token id를 string으로 변환. skip_special_token=True로 지정하면 speical token들을 제외
    - encode() : token string을 token id 의 리스트로 변환. add_special_tokens=False로 지정하면 token id로 변환할 때 special token들을 제외
        - padding을 통해서 padding token을 어떻게 추가할지 지정가능

- Dataset을 정의

In [9]:
# 챗봇 데이터를 처리하는 클래스를 만든다.
class ChatbotDataset(Dataset):
    def __init__(self, chats, max_len=40):  # 데이터셋의 전처리를 해주는 부분
        self._data = chats
        self.max_len = max_len
        self.q_token = Q_TKN
        self.a_token = A_TKN
        self.sent_token = SENT
        self.eos = EOS
        self.mask = MASK
        self.tokenizer = koGPT2_TOKENIZER

    def __len__(self):  # chatbotdata 의 길이를 리턴한다.
        return len(self._data)

    def __getitem__(self, idx):  # 로드한 챗봇 데이터를 차례차례 DataLoader로 넘겨주는 메서드
        turn = self._data.iloc[idx]
        q = turn["Q"]  # 질문을 가져온다.
        q = re.sub(r"([?.!,])", r" ", q)  # 구둣점들을 제거한다.

        a = turn["A"]  # 답변을 가져온다.
        a = re.sub(r"([?.!,])", r" ", a)  # 구둣점들을 제거한다.

        q_toked = self.tokenizer.tokenize(self.q_token + q + self.sent_token)
        q_len = len(q_toked)

        a_toked = self.tokenizer.tokenize(self.a_token + a + self.eos)
        a_len = len(a_toked)

        #질문의 길이가 최대길이보다 크면
        if q_len > self.max_len:
            a_len = self.max_len - q_len        #답변의 길이를 최대길이 - 질문길이
            if a_len <= 0:       #질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
                q_toked = q_toked[-(int(self.max_len / 2)) :]   #질문길이를 최대길이의 반으로 
                q_len = len(q_toked)
                a_len = self.max_len - q_len              #답변의 길이를 최대길이 - 질문길이
            a_toked = a_toked[:a_len]
            a_len = len(a_toked)

        #질문의 길이 + 답변의 길이가 최대길이보다 크면
        if q_len + a_len > self.max_len:
            a_len = self.max_len - q_len        #답변의 길이를 최대길이 - 질문길이
            if a_len <= 0:       #질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
                q_toked = q_toked[-(int(self.max_len / 2)) :]   #질문길이를 최대길이의 반으로 
                q_len = len(q_toked)
                a_len = self.max_len - q_len              #답변의 길이를 최대길이 - 질문길이
            a_toked = a_toked[:a_len]
            a_len = len(a_toked)

        # 답변 labels = [mask, mask, ...., mask, ..., <bos>,..답변.. <eos>, <pad>....]
        labels = [self.mask,] * q_len + a_toked[1:]

        # mask = 질문길이 0 + 답변길이 1 + 나머지 0
        mask = [0] * q_len + [1] * a_len + [0] * (self.max_len - q_len - a_len)
        # 답변 labels을 index 로 만든다.
        labels_ids = self.tokenizer.convert_tokens_to_ids(labels)
        # 최대길이만큼 PADDING
        while len(labels_ids) < self.max_len:
            labels_ids += [self.tokenizer.pad_token_id]

        # 질문 + 답변을 index 로 만든다.    
        token_ids = self.tokenizer.convert_tokens_to_ids(q_toked + a_toked)
        # 최대길이만큼 PADDING
        while len(token_ids) < self.max_len:
            token_ids += [self.tokenizer.pad_token_id]

        #질문+답변, 마스크, 답변
        return (token_ids, np.array(mask), labels_ids)

- 배치 데이터를 만들기 위해 collate_batch 함수를 정의

In [10]:
def collate_batch(batch):
    data = [item[0] for item in batch]
    mask = [item[1] for item in batch]
    label = [item[2] for item in batch]
    return torch.LongTensor(data), torch.LongTensor(mask), torch.LongTensor(label)


- Dataset 과 DataLoader를 정의

In [11]:
train_set = ChatbotDataset(Chatbot_Data, max_len=40)

#윈도우 환경에서 num_workers 는 무조건 0으로 지정, 리눅스에서는 2
train_dataloader = DataLoader(train_set, batch_size=32, num_workers=0, shuffle=True, collate_fn=collate_batch,)


- 데이터로더를 사용하여 테스트 데이터를 생성

In [12]:
print("start")
for batch_idx, samples in enumerate(train_dataloader):
    token_ids, mask, label = samples
    print("token_ids ====> ", token_ids)
    print("mask =====> ", mask)
    print("label =====> ", label)
print("end")

start
token_ids ====>  tensor([[    2, 10715, 18338,  ...,     3,     3,     3],
        [    2, 48397,  8711,  ...,     3,     3,     3],
        [    2, 17542, 12668,  ...,     3,     3,     3],
        ...,
        [    2, 18504, 20980,  ...,     3,     3,     3],
        [    2, 10715,  9511,  ...,     3,     3,     3],
        [    2, 10514,  7235,  ...,     3,     3,     3]])
mask =====>  tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])
label =====>  tensor([[9, 9, 9,  ..., 3, 3, 3],
        [9, 9, 9,  ..., 3, 3, 3],
        [9, 9, 9,  ..., 3, 3, 3],
        ...,
        [9, 9, 9,  ..., 3, 3, 3],
        [9, 9, 9,  ..., 3, 3, 3],
        [9, 9, 9,  ..., 3, 3, 3]])
token_ids ====>  tensor([[    2,  9815, 37655,  ...,     3,     3,     3],
        [    2, 10411, 39558,  ...,     3,     3,     3],
        [    2,  90

C:\Users\perso\AppData\Local\Temp\ipykernel_33284\4135955700.py:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:281.)
  return torch.LongTensor(data), torch.LongTensor(mask), torch.LongTensor(label)


#### 몇가지 PyTorch 함수

##### torch.repeat_interleave

- torch.repeat_interleave(input, repeats, dim=None, *, output_size=None)
- tensor의 element들을 반복. 즉 텐서를 반복 횟수만큼 복제.

- Parameters
    - input ( Tensor ) – 입력 텐서
    - repeats ( Tensor 또는 int ) – 각 요소의 반복 횟수
    - dim ( int , optional ) – 값을 반복 할 차원


In [13]:
x = torch.tensor([1, 2, 3])
x.repeat_interleave(2)     # x 덴서를 2번 반복합니다.

tensor([1, 1, 2, 2, 3, 3])

In [14]:
y = torch.tensor([[1, 2], [3, 4]])
torch.repeat_interleave(input=y,repeats=2)  # y 텐서를 2번 반복 합니다.

tensor([1, 1, 2, 2, 3, 3, 4, 4])

In [15]:
y = torch.tensor([[[1, 2, 3], [4, 5, 6], [7, 8, 9]]])
y.repeat_interleave(repeats=2, dim=1)   # y 텐서의 1차원을 2번 반복 합니다 

tensor([[[1, 2, 3],
         [1, 2, 3],
         [4, 5, 6],
         [4, 5, 6],
         [7, 8, 9],
         [7, 8, 9]]])

In [16]:
y.repeat_interleave(repeats=2, dim=2)        # y 텐서의 2차원을 2번 반복 합니다 

tensor([[[1, 1, 2, 2, 3, 3],
         [4, 4, 5, 5, 6, 6],
         [7, 7, 8, 8, 9, 9]]])

##### torch.Unsqueeze

- 지정된 위치에 1 차원 크기가 삽입 된 새 텐서를 반환
- squeeze함수는 차원이 1인 차원을 제거

In [23]:
x = torch.rand(3, 1, 20, 128)
x = x.squeeze() #[3, 1, 20, 128] -> [3, 20, 128]
x

tensor([[[0.3789, 0.3741, 0.1516,  ..., 0.9725, 0.0173, 0.3788],
         [0.1220, 0.1447, 0.1998,  ..., 0.2901, 0.0812, 0.6739],
         [0.5774, 0.3414, 0.4523,  ..., 0.2107, 0.7152, 0.5551],
         ...,
         [0.6725, 0.6418, 0.4965,  ..., 0.1499, 0.4942, 0.4380],
         [0.6540, 0.9736, 0.2048,  ..., 0.4150, 0.4610, 0.6332],
         [0.9484, 0.9349, 0.8612,  ..., 0.0467, 0.7207, 0.9133]],

        [[0.9357, 0.0923, 0.7617,  ..., 0.0893, 0.3393, 0.1721],
         [0.1864, 0.5566, 0.0832,  ..., 0.9067, 0.6743, 0.4585],
         [0.2921, 0.3851, 0.8352,  ..., 0.1814, 0.4427, 0.8829],
         ...,
         [0.7695, 0.5976, 0.1303,  ..., 0.7325, 0.6551, 0.2421],
         [0.5504, 0.4160, 0.9748,  ..., 0.7698, 0.2390, 0.0121],
         [0.9215, 0.9412, 0.2811,  ..., 0.6895, 0.9934, 0.4605]],

        [[0.4413, 0.1636, 0.3953,  ..., 0.1504, 0.8922, 0.7294],
         [0.2784, 0.4404, 0.3015,  ..., 0.6231, 0.1952, 0.8850],
         [0.2982, 0.0323, 0.8524,  ..., 0.8263, 0.0983, 0.

In [24]:
x = torch.rand(3, 20, 128)
x = x.unsqueeze(dim=1) #[3, 20, 128] -> [3, 1, 20, 128]
x

tensor([[[[0.5310, 0.4417, 0.4537,  ..., 0.0779, 0.2684, 0.1596],
          [0.5815, 0.0082, 0.6888,  ..., 0.3574, 0.7914, 0.1139],
          [0.6562, 0.2012, 0.5551,  ..., 0.9955, 0.4194, 0.8333],
          ...,
          [0.7961, 0.2014, 0.2954,  ..., 0.0859, 0.1757, 0.1711],
          [0.4581, 0.8075, 0.5157,  ..., 0.6236, 0.6912, 0.7753],
          [0.3458, 0.0212, 0.4691,  ..., 0.6106, 0.1767, 0.5967]]],


        [[[0.8574, 0.4596, 0.1329,  ..., 0.5862, 0.4278, 0.7391],
          [0.4603, 0.9988, 0.4680,  ..., 0.4421, 0.3899, 0.9763],
          [0.1135, 0.6545, 0.3319,  ..., 0.0035, 0.4971, 0.2206],
          ...,
          [0.8641, 0.4895, 0.6485,  ..., 0.9760, 0.6759, 0.3766],
          [0.4443, 0.7330, 0.4582,  ..., 0.0814, 0.4255, 0.9440],
          [0.3647, 0.4056, 0.5276,  ..., 0.3828, 0.1198, 0.9628]]],


        [[[0.3490, 0.8932, 0.9024,  ..., 0.6284, 0.7369, 0.1572],
          [0.0657, 0.5999, 0.2215,  ..., 0.5181, 0.9942, 0.6961],
          [0.3135, 0.5505, 0.6839,  ..

In [25]:
x = torch.tensor([1, 2, 3])
x.unsqueeze(1)

tensor([[1],
        [2],
        [3]])

In [26]:
y = torch.tensor([[1, 2], [3, 4]])
y.unsqueeze(1)

tensor([[[1, 2]],

        [[3, 4]]])

In [27]:
y = torch.tensor([[1, 2], [3, 4]])
y.unsqueeze(dim=2)

tensor([[[1],
         [2]],

        [[3],
         [4]]])

##### torch.where

- condition 에 따라 x 또는 y 에서 선택한 요소의 텐서를 반환

In [28]:
x = torch.tensor([[1,2,3], [-4,-5,-6], [7,8,9]])
y = torch.tensor([[11,12,13], [-14,-15,-16], [17,18,19]])
torch.where(x>0, x, y)     # x가 0보다 크다면 x 그렇지 않다면 y를 출력

tensor([[  1,   2,   3],
        [-14, -15, -16],
        [  7,   8,   9]])

In [29]:
x = torch.tensor([8, 4, 5, 6, 1, 9, 10, 1, 10, 7])
y = torch.tensor([2, 3, 3, 10, 10 , 4, 5, 6, 7 ,6])
torch.where( x>y, x, y)    # x가 y보다 크다면 x 그렇지 않다면 y를 출력

tensor([ 8,  4,  5, 10, 10,  9, 10,  6, 10,  7])

##### torch.transpose

- 주어진 차원 dim0 와 dim1 는 swap



In [30]:
x = torch.rand(2, 2, 2)
y = x.transpose(0, 2) # 0<->2 차원변경
print ("x=",x, "\ny=", y)

x= tensor([[[0.1667, 0.9258],
         [0.1271, 0.7285]],

        [[0.8495, 0.4017],
         [0.7475, 0.2672]]]) 
y= tensor([[[0.1667, 0.8495],
         [0.1271, 0.7475]],

        [[0.9258, 0.4017],
         [0.7285, 0.2672]]])


#### 일단 테스트

In [31]:
import torch
from transformers import GPT2LMHeadModel

In [32]:
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2", bos_token='</s>', eos_token='</s>', unk_token='<unk>', pad_token='<pad>', mask_token='<mask>') 
tokenizer.tokenize("안녕하세요. 한국어 GPT-2 입니다.😤:)l^o")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


['▁안녕',
 '하',
 '세',
 '요.',
 '▁한국어',
 '▁G',
 'P',
 'T',
 '-2',
 '▁입',
 '니다.',
 '😤',
 ':)',
 'l^o']

In [34]:
text = '근육이 커지기 위해서는'
input_ids = tokenizer.encode(text)
gen_ids = model.generate(torch.tensor([input_ids]),
                           max_length=250,
                           repetition_penalty=2.0,
                           pad_token_id=tokenizer.pad_token_id,
                           eos_token_id=tokenizer.eos_token_id,
                           bos_token_id=tokenizer.bos_token_id,
                           use_cache=True)
generated = tokenizer.decode(gen_ids[0,:].tolist())
print(generated)

근육이 커지기 위해서는 무엇보다 규칙적인 생활습관이 중요하다.
특히, 아침식사는 단백질과 비타민이 풍부한 과일과 채소를 많이 섭취하는 것이 좋다.
또한 하루 30분 이상 충분한 수면을 취하는 것도 도움이 된다.
아침 식사를 거르지 않고 규칙적으로 운동을 하면 혈액순환에 도움을 줄 뿐만 아니라 신진대사를 촉진해 체내 노폐물을 배출하고 혈압을 낮춰준다.
운동은 하루에 10분 정도만 하는 게 좋으며 운동 후에는 반드시 스트레칭을 통해 근육량을 늘리고 유연성을 높여야 한다.
운동 후 바로 잠자리에 드는 것은 피해야 하며 특히 아침에 일어나면 몸이 피곤해지기 때문에 무리하게 움직이면 오히려 역효과가 날 수도 있다.
운동을 할 때는 몸을 따뜻하게 하고 땀은 잘 흡수하도록 해야 한다.</d> 지난달 30일 오후 서울 종로구 세종로 정부중앙청사 별관. 이낙연 국무총리가 주재하던 국무회의가 열렸다.
국무위원들이 모두 참석한 가운데 열린 이날 회의에서는 ‘경제혁신 3개년 계획’ 등 주요 국정 현안에 대한 논의가 이뤄졌다.
김동연(사진) 경제부총리 겸 기획재정부 장관은 “올해는 우리 경제가 회복세를 보일 것으로 예상된다”며 “이런 상황에서 정부가 추진하는 각종 정책들을 차질 없이 추진하기 위해 최선을 다하겠다”고 말했다.
그는 이어 “우리 경제의 성장세가 지속되고 있는 만큼


#### Tokenizer

- 토크나이저는 모델에 대한 입력 준비를 담당

In [36]:
import numpy as np
import pandas as pd
import torch
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.core import LightningModule
from torch.utils.data import DataLoader, Dataset
from transformers.optimization import AdamW, get_cosine_schedule_with_warmup
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel
import re

In [37]:
Q_TKN = "<usr>"
A_TKN = "<sys>"
BOS = '</s>'
EOS = '</s>'
MASK = '<unused0>'
SENT = '<unused1>'
PAD = '<pad>'

In [38]:
koGPT2_TOKENIZER = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
            bos_token=BOS, eos_token=EOS, unk_token='<unk>',
            pad_token=PAD, mask_token=MASK) 
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [40]:
import urllib.request

urllib.request.urlretrieve(
    "https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv",
    filename="ChatBotData.csv",
)
Chatbot_Data = pd.read_csv("ChatBotData.csv")
# Test 용으로 300개 데이터만 처리한다.
Chatbot_Data = Chatbot_Data[:300]
Chatbot_Data.head()


,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [41]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_set = ChatbotDataset(Chatbot_Data, max_len=40)
#윈도우 환경에서 num_workers 는 무조건 0으로 지정, 리눅스에서는 2
train_dataloader = DataLoader(train_set, batch_size=32, num_workers=0, shuffle=True, collate_fn=collate_batch,)


In [45]:
device

device(type='cpu')

In [42]:
model.to(device)
model.train()


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=51200, bias=False)
)

In [43]:
learning_rate = 3e-5
criterion = torch.nn.CrossEntropyLoss(reduction="none")
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

epoch = 10
Sneg = -1e18



In [44]:

print ("start")
for epoch in range(epoch):
    for batch_idx, samples in enumerate(train_dataloader):
        optimizer.zero_grad()
        token_ids, mask, label = samples
        out = model(token_ids)
        out = out.logits      #Returns a new tensor with the logit of the elements of input
        mask_3d = mask.unsqueeze(dim=2).repeat_interleave(repeats=out.shape[2], dim=2)
        mask_out = torch.where(mask_3d == 1, out, Sneg * torch.ones_like(out))
        loss = criterion(mask_out.transpose(2, 1), label)
        # 평균 loss 만들기 avg_loss[0] / avg_loss[1] <- loss 정규화
        avg_loss = loss.sum() / mask.sum()
        avg_loss.backward()
        # 학습 끝
        optimizer.step()
print ("end")


We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


start
end


- 훈련에 8분 29초 소요(CPU)

- GPT2 모델 테스트

In [47]:
with torch.no_grad():
    while 1:
        q = input("user > ").strip()
        if q == "quit":
            break
        a = ""
        sent = ''
        while 1:
            input_ids = torch.LongTensor(koGPT2_TOKENIZER.encode(Q_TKN + q + SENT + sent + A_TKN + a)).unsqueeze(dim=0)
            pred = model(input_ids)
            pred = pred.logits
            gen = koGPT2_TOKENIZER.convert_ids_to_tokens(torch.argmax(pred, dim=-1).squeeze().numpy().tolist())[-1]
            if gen == EOS:
                break
            a += gen.replace("▁", " ")
        print("Chatbot > {}".format(a.strip()))


Chatbot > 좋은 아침이에요
Chatbot > 저도 바빠서 이야기할 시간이 부족했나봐요
Chatbot > 미리미리 충전해주세요
Chatbot > 가세요
Chatbot > 가족들과 상의해보세요
Chatbot > 너를 더 많이 아세요
Chatbot > 좋은 생각이에요
Chatbot > 거실이나 방에는 언제나 있는 자리니까요
Chatbot > 가세요
Chatbot > 가세요
Chatbot > 말해보세요
Chatbot > 하루가 또 가네요
Chatbot > 하루가 또 가네요
